## DCF Convexity

In [24]:
import os
import datetime as dt
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import plotly.express as px
import plotly.io as pio

pd.options.plotting.backend = "plotly"
# pio.templates.default = "plotly_dark"
pio.renderers.default = "notebook_connected"
px.defaults.width = 800
px.defaults.height = 500
plt.style.use("dark_background")
plt.rcParams["figure.dpi"] = 150


In [2]:
# Gordon growth
def gordon_growth(r, g, d1=100):
    return d1 / (r - g)


In [3]:
r = 0.1  # fix WACC=10%
g_range = np.linspace(0, 0.95 * r, 100)
vs = gordon_growth(r, g_range)
px.line(x=g_range, y=vs)


The Gordon growth model is convex in the growth rate (for a fixed WACC). This means that uncertainty in the growth rate *increases* the stock price.

## Gordon Growth Simulation

In [119]:
n = 200  # number of years, to approximate infinite sum
d0 = 100  # initial dividend
g_mean = 0.05
gs = g_mean * np.ones((n,))
gs[0] = 0
discount_factors = np.array([1 / (1 + r) ** i for i in range(n)])


In [120]:
fcfs = d0 * np.cumprod(1 + gs)
pv_fcfs = discount_factors * fcfs

In [121]:
px.line(pv_fcfs.cumsum())

In [122]:
def gordon_growth_sim(gs, r=0.10, d0=100):
    # Compute P
    fcfs = d0 * np.cumprod(1 + gs)
    discount_factors = np.array([1 / (1 + r) ** i for i in range(n)])
    pv_fcfs = discount_factors * fcfs
    return pv_fcfs.sum()


In [128]:
# Simulation: slow
n = 100  # years
it = 10_000  # sample size

stds = np.arange(0, 0.1, 0.01)
g_mean = 0.05
res = []
for std in stds:
    vs = []
    for i in range(it):
        gs = np.r_[[0], np.random.normal(g_mean, std, size=(n - 1,))]
        vs.append(gordon_growth_sim(gs))
    res.append([std, np.mean(vs), np.std(vs) / np.sqrt(it)])

res_df = pd.DataFrame(res, columns=["std", "v", "v_err"])
res_df


,std,v,v_err
0,0.00,2179.006538,0.000000
1,0.01,2178.321042,0.651457
2,0.02,2180.090978,1.318248
3,0.03,2182.476831,1.993827
4,0.04,2179.654187,2.649488
5,0.05,2178.962647,3.331370
6,0.06,2182.741775,4.031885
7,0.07,2170.593152,4.696734
8,0.08,2181.554592,5.449680
9,0.09,2170.652313,6.101081


In [141]:
# Same simulation but 6x faster
p = 11
n = 200  # years
it = 100_000  # sample size

stds = np.linspace(0, 0.2, p)
g_mean = 0.03
means = g_mean * np.ones((p,))

vs_its = []
for i in range(it):

    xs = np.random.normal(means, stds, size=(n - 1, p))
    g_arr = xs.T  # each row is a growth process with different std
    g_arr = np.hstack([np.zeros((p, 1)), g_arr])
    fcfs = d0 * (1 + g_arr).cumprod(axis=1)
    discount_factors = np.array([1 / (1 + r) ** i for i in range(n)])
    pv_fcfs = fcfs * discount_factors
    vs = pv_fcfs.sum(axis=1)
    vs_its.append(vs)
v = np.stack(vs_its).mean(axis=0)
v_err = np.stack(vs_its).std(axis=0) / np.sqrt(it)
res_df = pd.DataFrame([stds, v, v_err], index=["stds", "v", "v_err"]).T
res_df


,stds,v,v_err
0,0.00,1571.425515,7.555448e-12
1,0.02,1571.603252,2.567039e-01
2,0.04,1571.590758,5.186156e-01
3,0.06,1571.564494,7.856040e-01
4,0.08,1570.081689,1.052956e+00
5,0.10,1570.717167,1.332452e+00
6,0.12,1569.865923,1.615251e+00
7,0.14,1568.759868,1.922869e+00
8,0.16,1574.509206,2.269119e+00
9,0.18,1568.923669,2.580929e+00


In [142]:
fig = px.scatter(res_df, x="stds", y="v", error_y="v_err")
# fig.update_layout(yaxis_range=[21, 2300])
fig


While the Gordon growth model is convex to $g$, there is little convexity to any given $g_t$. In fact, variance drag should come into play.

## 2-stage DCF

In [56]:
def growth_stage(cf, g0, n=5, r=0.10):
    return cf / (r - g0) * (1 - ((1 + g0) / (1 + r)) ** (n + 1)) * (1 + r)


def perpetuity(cf, g0, n=5, r=0.10, g=0.025):
    v = cf * (1 + g0) ** n * (1 + g)
    discount_factor = 1 / (1 + r) ** n
    return v / (r - g) * discount_factor


def dcf_2_stage(cf, g0, n=5, r=0.10, g=0.025):
    return growth_stage(cf, g0, n, r) + perpetuity(cf, g0, n, r, g)


In [57]:
# Check it gives same result as spreadsheet
print(dcf_2_stage(10, 0.30, n=10, r=0.1, g=0.025))
print(dcf_2_stage(10, 0.25, n=10, r=0.1, g=0.025))
print(dcf_2_stage(10, 0.35, n=10, r=0.1, g=0.025))


1016.8667312023636
716.6100383285764
1434.0380259760666


In [58]:
# Convexity to growth
gs = np.linspace(0, 0.5, 100)
vs0 = growth_stage(10, gs, n=10, r=0.1)
vs1 = perpetuity(10, gs, n=10, r=0.1)
vs = dcf_2_stage(10, gs, n=10, r=0.1, g=0.025)
df = pd.DataFrame({"g0": gs, "growth": vs0, "perpetuity": vs1, "pv": vs})
# fig = px.line(df, x="g0", y=["pv", "growth", "perpetuity"])
fig = px.line(df, x="g0", y="pv")
fig.add_vline(x=0.3, line_dash="dash")
fig.update_layout(xaxis_tickformat=".1%")


In [59]:
fig.write_html("dcf_convexity.html", include_plotlyjs="/assets/plotly.min.js")


In [62]:
# Convexity to discount rate
rs = np.linspace(0.05, 0.29, 100)
vs = dcf_2_stage(10, 0.3, n=10, r=rs)
fig = px.line(x=rs, y=vs)
fig.add_vline(x=0.1, line_dash="dash")
fig.update_layout(xaxis_title="r", yaxis_title="PV")
fig.update_layout(xaxis_tickformat=".1%")
fig.show()


In [63]:
fig.write_html("dcf_convexity_wacc.html", include_plotlyjs="/assets/plotly.min.js")


In [94]:
# Expected intrinsic value as a function of uncertainty in the growth rate
g0_mean = 0.3
stds = np.linspace(0, 0.4, 41)
it = 100_000

res = []
for std in stds:

    g_samples = np.random.normal(g0_mean, std, size=(it,))
    v_samples = dcf_2_stage(100, g_samples, n=10, r=0.1)
    res.append([std, np.mean(v_samples), np.std(v_samples) / np.sqrt(it)])

res_df = pd.DataFrame(res, columns=["std", "PV", "PV_err"])
# res_df.iloc[:, 1:] /= res_df.iloc[0, 1]
fig = px.line(res_df, x="std", y="PV", error_y="PV_err")
fig.show()


In [21]:
fig.write_html("convexity_value.html", include_plotlyjs="/assets/plotly.min.js")


## 2-stage DCF: cash flow volatility

In [84]:
def perpetuity_last(cf_last, n=5, r=0.10, g=0.025):
    v = cf_last * (1 + g)
    discount_factor = 1 / (1 + r) ** n
    return v / (r - g) * discount_factor


def dcf_stoch(cf, gs, n=5, r=0.10):
    # Compute P
    fcfs = cf * np.cumprod(1 + gs)
    discount_factors = np.array([1 / (1 + r) ** i for i in range(n + 1)])
    pv_fcfs = discount_factors * fcfs
    return pv_fcfs.sum() + perpetuity_last(fcfs[-1], n, r, g=0.025)


In [85]:
# check it gives same result as spreadsheet in case of zero vol
n = 10
gs = np.r_[[0], np.random.normal(0.3, 0, size=(n,))]
dcf_stoch(10, gs, n=10, r=0.1)


1016.8667312023642

In [116]:
# Simulation: slow
n = 10  # years
it = 10_000  # sample size

stds = np.arange(0, 1, 0.1)
g_mean = 0.3

res = []
for std in stds:
    vs = []
    for i in range(it):
        gs = np.r_[[0], np.random.normal(0.3, std, size=(n,))]
        vs.append(dcf_stoch(10, gs, n, r=0.1))
    res.append([std, np.mean(vs), np.std(vs) / np.sqrt(it)])

res_df = pd.DataFrame(res, columns=["cf_std", "PV", "PV_err"])

In [117]:
fig = px.scatter(res_df, x="cf_std", y="PV", error_y="PV_err")
fig.update_layout(xaxis_tickformat=".0%")


In [118]:
fig.write_html("cf_std.html", include_plotlyjs="/assets/plotly.min.js")
